In [1]:
import requests
import pandas as pd

In [2]:
url = "https://api.spacexdata.com/v4/launches/upcoming"


In [3]:
results = requests.get(url).json()

In [5]:
space=[]
for result in results:
    #print(results["flight_number"])
    space.append({"flight_id": result["id"],
                "launch_date": result["date_local"],
                "launch_pad": result["launchpad"]})

In [6]:
space_df=pd.DataFrame(space)

In [7]:
space_df

,flight_id,launch_date,launch_pad
0,60428aafc041c16716f73cd7,2021-03-22T18:19:00-04:00,5e9e4501f509094ba4566f84
1,60428ac4c041c16716f73cd8,2021-02-28T19:00:00-05:00,5e9e4501f509094ba4566f84
2,5fe3af58b3467846b324215f,2021-04-22T06:11:00-04:00,5e9e4502f509094188566f88
3,5fe3af43b3467846b324215e,2021-04-30T17:00:00-07:00,5e9e4502f509092b78566f87
4,600f9b6d8f798e2a4d5f979f,2021-05-31T20:00:00-04:00,5e9e4501f509094ba4566f84
5,5fe3af84b3467846b3242161,2021-05-31T20:00:00-04:00,5e9e4502f509094188566f88
6,5fe3afc1b3467846b3242164,2021-05-31T20:00:00-04:00,5e9e4501f509094ba4566f84
7,5fe3af6db3467846b3242160,2021-03-31T20:00:00-04:00,5e9e4501f509094ba4566f84
8,5fe3af99b3467846b3242162,2021-06-30T20:00:00-04:00,5e9e4502f509094188566f88
9,5eb87d4effd86e000604b390,2021-06-30T20:00:00-04:00,5e9e4501f509094ba4566f84


In [12]:
url_b="https://api.spacexdata.com/v4/launchpads"
results_b = requests.get(url_b).json()
results_b

[{'name': 'VAFB SLC 3W',
  'full_name': 'Vandenberg Air Force Base Space Launch Complex 3W',
  'locality': 'Vandenberg Air Force Base',
  'region': 'California',
  'timezone': 'America/Los_Angeles',
  'latitude': 34.6440904,
  'longitude': -120.5931438,
  'launch_attempts': 0,
  'launch_successes': 0,
  'rockets': ['5e9d0d95eda69955f709d1eb'],
  'launches': [],
  'details': "SpaceX's original west coast launch pad for Falcon 1. It was used in a static fire test but was never employed for a launch, and was abandoned due to range scheduling conflicts arising from overflying other active pads.",
  'status': 'retired',
  'id': '5e9e4501f5090910d4566f83'},
 {'name': 'CCSFS SLC 40',
  'full_name': 'Cape Canaveral Space Force Station Space Launch Complex 40',
  'locality': 'Cape Canaveral',
  'region': 'Florida',
  'timezone': 'America/New_York',
  'latitude': 28.5618571,
  'longitude': -80.577366,
  'launch_attempts': 67,
  'launch_successes': 65,
  'rockets': ['5e9d0d95eda69973a809d1ec'],
 

In [17]:
pads = []
#results
for result in results_b:
    pads.append({"Name":result["full_name"],
                 "launch_pad": result["id"],
                 "lat": result["latitude"],
                 "long": result["longitude"]})

In [18]:
pads

[{'Name': 'Vandenberg Air Force Base Space Launch Complex 3W',
  'launch_pad': '5e9e4501f5090910d4566f83',
  'lat': 34.6440904,
  'long': -120.5931438},
 {'Name': 'Cape Canaveral Space Force Station Space Launch Complex 40',
  'launch_pad': '5e9e4501f509094ba4566f84',
  'lat': 28.5618571,
  'long': -80.577366},
 {'Name': 'SpaceX South Texas Launch Site',
  'launch_pad': '5e9e4502f5090927f8566f85',
  'lat': 25.9972641,
  'long': -97.1560845},
 {'Name': 'Kwajalein Atoll Omelek Island',
  'launch_pad': '5e9e4502f5090995de566f86',
  'lat': 9.0477206,
  'long': 167.7431292},
 {'Name': 'Vandenberg Air Force Base Space Launch Complex 4E',
  'launch_pad': '5e9e4502f509092b78566f87',
  'lat': 34.632093,
  'long': -120.610829},
 {'Name': 'Kennedy Space Center Historic Launch Complex 39A',
  'launch_pad': '5e9e4502f509094188566f88',
  'lat': 28.6080585,
  'long': -80.6039558}]

In [19]:
pads_df=pd.DataFrame(pads)

In [20]:
launch_location=space_df.merge(pads_df,on="launch_pad")

In [21]:
launch_location

,flight_id,launch_date,launch_pad,Name,lat,long
0,60428aafc041c16716f73cd7,2021-03-22T18:19:00-04:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
1,60428ac4c041c16716f73cd8,2021-02-28T19:00:00-05:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
2,600f9b6d8f798e2a4d5f979f,2021-05-31T20:00:00-04:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
3,5fe3afc1b3467846b3242164,2021-05-31T20:00:00-04:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
4,5fe3af6db3467846b3242160,2021-03-31T20:00:00-04:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
5,5eb87d4effd86e000604b390,2021-06-30T20:00:00-04:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
6,5fe3af58b3467846b324215f,2021-04-22T06:11:00-04:00,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956
7,5fe3af84b3467846b3242161,2021-05-31T20:00:00-04:00,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956
8,5fe3af99b3467846b3242162,2021-06-30T20:00:00-04:00,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956
9,5fe3b11eb3467846b324216c,2021-08-17T20:00:00-04:00,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956


In [23]:
launch_location[['launch_date', 'launch_time']] = launch_location['launch_date'].str.split('T', expand=True)
launch_location

,flight_id,launch_date,launch_pad,Name,lat,long,launch_time
0,60428aafc041c16716f73cd7,2021-03-22,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366,18:19:00-04:00
1,60428ac4c041c16716f73cd8,2021-02-28,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366,19:00:00-05:00
2,600f9b6d8f798e2a4d5f979f,2021-05-31,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366,20:00:00-04:00
3,5fe3afc1b3467846b3242164,2021-05-31,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366,20:00:00-04:00
4,5fe3af6db3467846b3242160,2021-03-31,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366,20:00:00-04:00
5,5eb87d4effd86e000604b390,2021-06-30,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366,20:00:00-04:00
6,5fe3af58b3467846b324215f,2021-04-22,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956,06:11:00-04:00
7,5fe3af84b3467846b3242161,2021-05-31,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956,20:00:00-04:00
8,5fe3af99b3467846b3242162,2021-06-30,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956,20:00:00-04:00
9,5fe3b11eb3467846b324216c,2021-08-17,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956,20:00:00-04:00


In [26]:
weath_url="http://api.weatherbit.io/v2.0/forecast/daily?key=24bb5c9cb97c4a06a04d2a1bba2b75aa&lat=28.561857&lon=-80.577366"
weath_results = requests.get(weath_url).json()
weath_results

{'data': [{'moonrise_ts': 1616077867,
   'wind_cdir': 'S',
   'rh': 84,
   'pres': 1014.29,
   'high_temp': 25.8,
   'sunset_ts': 1616110394,
   'ozone': 280.49,
   'moon_phase': 0.277524,
   'wind_gust_spd': 13.3984,
   'snow_depth': 0,
   'clouds': 32,
   'ts': 1616040060,
   'sunrise_ts': 1616066857,
   'app_min_temp': 21.1,
   'wind_spd': 5.70403,
   'pop': 10,
   'wind_cdir_full': 'south',
   'slp': 1014.29,
   'moon_phase_lunation': 0.19,
   'valid_date': '2021-03-18',
   'app_max_temp': 26.1,
   'vis': 24.096,
   'dewpt': 19.5,
   'snow': 0,
   'uv': 6.97224,
   'weather': {'icon': 'c02d', 'code': 802, 'description': 'Scattered clouds'},
   'wind_dir': 186,
   'max_dhi': None,
   'clouds_hi': 6,
   'precip': 0.25,
   'low_temp': 18.8,
   'max_temp': 25.9,
   'moonset_ts': 1616041209,
   'datetime': '2021-03-18',
   'temp': 22.5,
   'min_temp': 20.5,
   'clouds_mid': 0,
   'clouds_low': 29},
  {'moonrise_ts': 1616166471,
   'wind_cdir': 'S',
   'rh': 75,
   'pres': 1013.98,
   'h

In [46]:
weather = []
params = {
    "key": "24bb5c9cb97c4a06a04d2a1bba2b75aa"
}

weath_url="http://api.weatherbit.io/v2.0/forecast/daily"

for index, row in launch_location.iterrows():
    params["lat"]= row["lat"]
    params["lon"]= row["long"]
    address= requests.get(weath_url, params=params)
    address_j=address.json()
    
    for result in address_j["data"]:
        post= {"lat": address_j["lat"],
              "long":address_j["lon"],
              "Valid_date":result["valid_date"],}
        weather.append(post)
print(post)


{'lat': 34.63, 'long': -120.61, 'Valid_date': '2021-04-01'}


http://api.weatherbit.io/v2.0/forecast/daily?key=24bb5c9cb97c4a06a04d2a1bba2b75aa&lat=28.5618571&lon=-80.577366


In [ ]:
weather = []
#results
for result in results_b:
    weather.append({"Valid_date":result["data"]["valid_date"],
                    "Ave_Temp": result["data"]["temp"],
                    "Max_Temp":result["data"]["max_temp"],
                    "Min_Temp": result["data"]["min_temp"],
                    "H_Level_Cloud":result["data"]["clouds_hi"],
                    "Wind_Gust_Spd":result["data"]["wind_gust_spd"],
                    "weather_condition":result["data"]["weather"]["description"],
                    "Lat":result["lat"],
                    "Long":result["lon"]})